In [3]:
#install latest Chrome driver to your machine https://sites.google.com/chromium.org/driver/
#install Beautiful Soup 
#install selenium

#imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
import time
import json 


#base product URL
URL = "https://e-katanalotis.gov.gr/product/"
min_id = 0
max_id = 1353

#creates a list [min_id...max_id-1]
productids = range(min_id, max_id)

#final JSON
results = {}
results['fetch_date']=int(time.time())
results['data']=[]

#get product information
for productid in productids:
    print(productid)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    # executable_path param is where the Chrome driver is installed
    browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')
    browser.get(URL+str(productid))

    #get dates from price chart
    try:
        date_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.category)')
    except exceptions.JavascriptException as e:
        continue
    #append year to date labels
    date_data = [s +'/2022' for s in date_data]

    #get prices from price chart
    try:
        price_data = browser.execute_script('return Highcharts.charts[0].series[0].data.map(x => x.y)')
    except exceptions.JavascriptException as e:
        continue
    #find product name
    html = browser.page_source
    soup = BeautifulSoup(html, features="html5lib")

    #test html rendering
    # print(soup.prettify())

    try:
        # pname = soup.find('p', attrs={'class':'product-name'}).get_text()
        pname = soup.find('p', attrs={'class':'product-category-name'}).get_text() 
    except AttributeError as e:
        continue
    browser.quit()

    #transform dates

    for i in range(len(date_data)-1):
        dparts = date_data[i].split("/")
        newd = '2022-'+dparts[1]+'-'+dparts[0]
        date_data[i]=newd
        
    #create a result object
    result = {}
    result['id']=productid
    result['name']=pname
    result['prices']=[]
    for i in range(len(date_data)-1):
        result['prices'].append({'date':date_data[i], 'price':price_data[i]})
    
    #append it to the list
    results['data'].append(result)

print('Done')

0


/tmp/ipykernel_4143/2452124923.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(options=options, executable_path='/usr/bin/chromedriver')


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115


KeyboardInterrupt: 

In [ ]:
#inspect final JSON
json_object = json.dumps(results, indent = 2, ensure_ascii=False).encode('utf8')
print(json_object.decode())

{
  "fetch_date": 1670318130,
  "data": []
}


In [ ]:
#save final JSON to file
with open("output_"+str(productids[0])+"_"+str(productids[len(productids)-1])+"_"+str(results['fetch_date'])+".json", "w", encoding='utf-8') as outfile:
    json.dump(results, outfile, ensure_ascii=False ,indent=2)